Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from yeast8model import Yeast8Model, compare_fluxes

# Construct models of cells of interest, optimise

In [ ]:
wt = Yeast8Model("./models/ecYeastGEM_batch.xml")
wt.solution = wt.optimize()

BY4741 = Yeast8Model("./models/ecYeastGEM_batch.xml")
BY4741.make_auxotroph("BY4741")
BY4741.solution = BY4741.optimize()

zwf1 = Yeast8Model("./models/ecYeastGEM_batch.xml")
zwf1.make_auxotroph("BY4741")
zwf1.knock_out_list(["YNL241C"])
zwf1.solution = zwf1.optimize()

tsa2 = Yeast8Model("./models/ecYeastGEM_batch.xml")
tsa2.make_auxotroph("BY4742")
tsa2.knock_out_list(["YDR453C"])
tsa2.solution = tsa2.optimize()

# Ablate

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type')
plt.show()

In [ ]:
BY4741.ablation_result = BY4741.ablate()

fig, ax = plt.subplots()
BY4741.ablation_barplot(ax)
ax.set_title('BY4741 (in supplemented media)')
plt.show()

In [ ]:
zwf1.ablation_result = zwf1.ablate()
fig, ax = plt.subplots()
zwf1.ablation_barplot(ax)
ax.set_title('zwf1$\Delta$ BY4741 (in supplemented media)')
plt.show()

In [ ]:
tsa2.ablation_result = tsa2.ablate()
fig, ax = plt.subplots()
tsa2.ablation_barplot(ax)
ax.set_title('tsa2$\Delta$ BY4742 (in supplemented media)')
plt.show()

In [ ]:
wt.ablation_result

In [ ]:
BY4741.ablation_result

In [ ]:
zwf1.ablation_result

In [ ]:
tsa2.ablation_result

## Add amino acids to minimal media

In [ ]:
amino_exch_list = [
    'r_1873', 'r_1879', 'r_1880', 'r_1881', 'r_1883',
    'r_1889', 'r_1891', 'r_1810', 'r_1893', 'r_1897',
    'r_1899', 'r_1900', 'r_1902', 'r_1903', 'r_1904',
    'r_1906', 'r_1911', 'r_1912', 'r_1913', 'r_1914',
]

wt.reset()
wt.add_media_components(amino_exch_list)

In [ ]:
for reaction_id in wt.model.medium.keys():
    print(wt.model.reactions.get_by_id(reaction_id).name)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with all amino acids supplemented')
plt.show()

> You can see that it takes less time to make proteins.

## Add nucleotides to minimal media

In [ ]:
nucl_exch_list = [
    'r_1639', 'r_1705', 'r_1818', 'r_2090'
]

wt.reset()
wt.add_media_components(nucl_exch_list)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with all NTPs supplemented')
plt.show()

In [ ]:
wt.ablation_result

> It's difficult to see, but the original RNA flux was 1.789725 and with nucleotides added, it is now 2.348641.  Other fluxes are largely unchanged, though DNA is slightly faster.

## Add deoxyribonucleotides to media

In [ ]:
deoxnucl_exch_list = [
    'r_1643', 'r_1702', 'r_1820', 'r_2073'
]

wt.reset()
wt.add_media_components(deoxnucl_exch_list)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with all dNTPs supplemented')
plt.show()

In [ ]:
wt.ablation_result

> DNA: 2.364069 --> 2.416639.  Curiously, RNA: --> 2.387195

## Different carbon sources

Pyruvate

In [ ]:
wt.reset()
wt.add_media_components(['r_2033'])
wt.remove_media_components(['r_1714', 'r_1714_REV'])
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with pyruvate as carbon source')
plt.show()

In [ ]:
wt.ablation_result

# Compare fluxes

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, zwf1)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, tsa2)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
for reaction_id in BY4741.model.medium.keys():
    print(BY4741.model.reactions.get_by_id(reaction_id).name)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, wt)

In [ ]:
diff_fluxes_sorted

# Effect of glucose uptake on growth rate

Optimise wt with glucose unconstrained.

In [ ]:
wt.reset()
wt.solution = wt.optimize()

Get saturated glucose uptake:

In [ ]:
wt.solution['r_1714_REV']

> You should get 17.9 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup>.  This agrees with Elsemman et al. (2022): they predict saturation at 18.6 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup> and report a range of 16 ~ 19 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup> from the literature (Blank et al., 2004).

Sweep, across all strains.

In [ ]:
# positive values because i want it increasing in a plot
glc_exch_rates = np.linspace(0, 18.6, 100)

wt_growthrate = []
BY4741_growthrate = []
zwf1_growthrate = []
tsa2_growthrate = []

def get_gr_from_glc_rate(ymodel, growthratelist):
    # negative due to FBA conventions re exchange reactions
    ymodel.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
    # positive due to FBA conventions re reversible reaction
    ymodel.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
    ymodel.solution = ymodel.optimize()
    growthratelist.append(ymodel.solution.fluxes["r_2111"])

for glc_exch_rate in glc_exch_rates:
    get_gr_from_glc_rate(wt, wt_growthrate)
    get_gr_from_glc_rate(BY4741, BY4741_growthrate)
    get_gr_from_glc_rate(zwf1, zwf1_growthrate)
    get_gr_from_glc_rate(tsa2, tsa2_growthrate)

print('optimisations done')

In [ ]:
fig, ax = plt.subplots()
ax.plot(glc_exch_rates, wt_growthrate, label='prototroph (wild type)')
ax.plot(glc_exch_rates, BY4741_growthrate, label='BY4741, in supplemented media')
ax.plot(glc_exch_rates, zwf1_growthrate, label='BY4741 zwf1, in supplemented media')
ax.plot(glc_exch_rates, tsa2_growthrate, label='BY4742 tsa2, in supplemented media')
ax.set_xlim((0,8))
ax.set_xlabel('Glucose exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of glucose exchange rate on growth rate')
ax.legend()

> Results from wild-type are similar to with Elsemman et al. (2022), figure 2B. Saturation (or near it) seems to be reached at about 8 mmol/gDW/h.

In [ ]:
doubling_times = np.log(2) / np.array(growthrate)

fig, ax = plt.subplots()
ax.plot(glc_exch_rates, doubling_times)
ax.set_xlim((1,8))
ax.set_ylim((0,5))
ax.set_xlabel('Glucose exchange rate (mmol/gDW/h)')
ax.set_ylabel('Doubling time (h)')
ax.set_title('Effect of glucose exchange rate on doubling time')